# ドロップアウト (Dropout)

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# 学習データと検証用データに分割
from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="data")
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、ノード数が128の隠れ層を5層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

#  トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=500, verbose=0)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

## ドロップアウト (Dropout)

ドロップアウトは、学習時に、ランダムにノードを消すという操作をすることで、表現能力を抑制する手法です。
この操作は、アンサンブル学習を擬似的に行っていると見ることもできます。

この操作もKerasの関数を用いることで簡単に実装することができます。

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout

```python
Dropout(rate=0.3)
```
ここで、消すノードの割合を`rate`で指定します。0だと、ノードは全く消されません。1.0だと、全てのノードが消されます。

ここでは、MLPの各層の後にドロップアウト層を入れることにします。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dropout(rate=0.3),  # 30%の確率でノードの値を0にする。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

#  トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=500, verbose=0)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

トレーニングデータに対する誤差関数が大きくふらついていることがわかります。
これは、消されるノードが毎回ランダムに選ばれるためです。
一方で、検証用データではばらつきは比較的小さくなっています。
これは、Kerasの実装では、検証用データを使ってモデルの予測をする際は、全てのノードを使うようになっているためです。

ドロップアウトの`rate`の値を変えたり、ドロップアウトの位置を変えると、モデルの予測はどのように変化するでしょうか？